In [1]:
import os,sys,tqdm
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import *

from collections import Counter
import pandas as pd
import shutil
import pickle
import gc
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def basic_statistics(all_length):
    '''
    input: length list of elements e.g.[1,1,1,3,5,9,4,2,1,3,54,78,5...]
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

In [3]:
max_words = 8352#8352 #Top most frequent words to consider. Any less frequent word will appear as oov_char value in the sequence data.
max_length = 360#360

In [4]:
word_index = reuters.get_word_index()
print('all_words#:',len(word_index))
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,maxlen=max_length,
                                                         test_split=0.2,seed=830913)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

all_words#: 30979
8260 train sequences
2066 test sequences


In [13]:
# train_len = [len(x) for x in x_train]
# test_len = [len(x) for x in x_test]
# all_len = train_len
# all_len.extend(test_len)
# basic_statistics(all_len)

,length
mean,145.964197
std,145.878476
mode,17.000000
min,2.000000
q1,60.000000
median,95.000000
q3,180.000000
max,2376.000000
iqr,120.000000
outlier,360.000000


In [15]:
# df = pd.DataFrame(all_len)
# df.to_excel('./results/length_dist.xlsx', header=False, index=False)
# df

In [21]:
# train_words = []
# for x in x_train:
#     train_words.extend(x)
# test_words = []
# for x in x_test:
#     test_words.extend(x)
# all_words = train_words
# all_words.extend(test_words)
# all_statistcs = Counter(all_words)
# all_statistcs

Counter({1: 11228,
         53: 4213,
         352: 647,
         26: 8451,
         14: 15015,
         279: 801,
         39: 5818,
         72: 3091,
         4497: 26,
         18: 11039,
         83: 2597,
         5291: 21,
         88: 2381,
         5397: 20,
         11: 20141,
         3412: 37,
         19: 10755,
         151: 1363,
         230: 962,
         831: 253,
         15: 13329,
         165: 1232,
         318: 707,
         3780: 33,
         124: 1676,
         1527: 117,
         1424: 128,
         35: 6588,
         5302: 20,
         12: 16668,
         17: 11191,
         486: 459,
         341: 663,
         142: 1466,
         255: 870,
         219: 997,
         429: 528,
         68: 3363,
         146: 1402,
         252: 882,
         191: 1098,
         15448: 3,
         3631: 35,
         2283: 65,
         71: 3120,
         10: 29581,
         342: 660,
         49: 4565,
         1977: 80,
         324: 695,
         27: 8311,
         9222: 

In [28]:
# df = pd.DataFrame.from_dict(dict(all_statistcs), orient = 'index')
# df.to_excel('./results/words_dist2.xlsx', header=False, index=True)
# df

,0
1,11228
53,4213
352,647
26,8451
14,15015
...,...
24452,1
18567,2
27222,1
26864,1


In [5]:
trainX = tf.keras.preprocessing.sequence.pad_sequences(x_train,maxlen=max_length,padding='post',value=0)
testX = tf.keras.preprocessing.sequence.pad_sequences(x_test,maxlen=max_length,padding='post',value=0)
print(trainX.shape, testX.shape)

(8260, 360) (2066, 360)


In [14]:
hidden_dim = 256
do = 0.1

## Graph execution
### Embedder

In [42]:
int_id = Input(shape=(max_length,), dtype='int32', name='int_ids') # 輸入的api funvtion name ID
int_ids = Masking(mask_value=0)(int_id)
sent_emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')(int_ids) 

### Encoder

In [43]:
rnn = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)(sent_emb)
rnn = BatchNormalization(name='bn')(rnn)


### Filter

In [44]:
fil = TimeDistributed(Dense(1,activation='sigmoid',
                             name='filter_out'),name='TD2')(rnn)

### Classfier

In [47]:
mul = Multiply()([fil,sent_emb])
clf = LSTM(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm')(mul)
clf = BatchNormalization(name='bn3')(clf)
clf = Dense(max(y_train)+1,activation='softmax',name='clf')(clf)

## Compile

In [48]:
model = Model(inputs=int_id, outputs = clf)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
int_ids (InputLayer)            [(None, 358)]        0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 358)          0           int_ids[0][0]                    
__________________________________________________________________________________________________
glove_emb (Embedding)           (None, 358, 128)     1280000     masking_1[0][0]                  
__________________________________________________________________________________________________
common_extract (GRU)            (None, 358, 64)      37248       glove_emb[0][0]                  
______________________________________________________________________________________________

In [53]:
# loss
import keras.backend as K
def custom_objective(layer):
    return K.sum(layer.output)
#     return K.sum(layer.output)
# kk = tf.keras.backend.ea
model.compile(loss=custom_objective(model.get_layer(name='TD2')),optimizer='adam')

Using TensorFlow backend.


OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

## Eager Execution

In [52]:
# whole model
do = 0
init = tensorflow.keras.initializers.Ones()
class base_model(Model):
    def __init__(self):
        super(base_model, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        self.rnn1 = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.bn1 = BatchNormalization(name='bn1')
        self.fil = Dense(1,activation='hard_sigmoid',kernel_initializer=init,bias_initializer=init,name='filter_out')
        #self.fil = TimeDistributed(Dense(1,activation='sigmoid', name='filter_out'),name='TD2')
        self.mul = Multiply()
        self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = LSTM(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')
    def transform(self,x):
        return tf.math.round(x)
    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.rnn1(x1)
        x = self.bn1(x)
        y = self.fil(x)
        y1 = self.transform(y)
        x2 = self.mul([y1,x1])
        x = self.rnn2(x2) #x
        x = self.bn2(x)
        y2 = self.out(x)
        return y,y1,y2
        #return y,y1,y2,x2
        
model = base_model()

In [100]:
# partial1 model
init = tensorflow.keras.initializers.Zeros()
def onezero(x):
    beta = 0.6 #0.1~1
    z = tf.where(x>=1.0, x - x + 1.0, x)
    y = tf.where(z<=0.0, z - z + 0.0, beta*z)
    return y

class base_model_1(Model):
    def __init__(self):
        super(base_model_1, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        self.rnn1 = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.att = Attention(name='selfatt')
        self.bn1 = BatchNormalization(name='bn1')
        #self.fil = Dense(1,activation='sigmoid',name='filter_out')
        self.fil = TimeDistributed(Dense(1,activation=onezero, name='filter_out'),name='TD2') #relu/linear/step function

    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.att([x1,x1])
        #x = self.rnn1(x1)
        #x = self.bn1(x)
        y = self.fil(x)
        return x1,y

model1 = base_model_1()

In [101]:
# partial2 model
class base_model_2(Model):
    def __init__(self):
        super(base_model_2, self).__init__()
        self.mul = Multiply()
        self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = GRU(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')

    def call(self,x1,y1):
        x2 = self.mul([y1,x1])
        x = self.rnn3(x2) #x
        x = self.bn2(x)
        y2 = self.out(x)
        return y2
    
model2 = base_model_2()

In [162]:
# TEST
# x = tf.random.uniform((1, max_length))
# out1,out2,out3,out4 = model(x)
# out1, out4

<tf.Tensor: id=264354, shape=(1, 358, 1), dtype=float32, numpy=
array([[[0.50008374],
        [0.50039524],
        [0.50061977],
        [0.50072163],
        [0.5007394 ],
        [0.50071514],
        [0.5006769 ],
        [0.50063914],
        [0.50060797],
        [0.5005844 ],
        [0.50056773],
        [0.50055635],
        [0.5005488 ],
        [0.5005439 ],
        [0.50054073],
        [0.50053877],
        [0.5005376 ],
        [0.5005368 ],
        [0.5005363 ],
        [0.500536  ],
        [0.50053585],
        [0.5005358 ],
        [0.5005357 ],
        [0.50053567],
        [0.50053567],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5005356 ],
        [0.5

In [102]:
batch_size = 128 #,reshuffle_each_iteration=True
train_ds = tf.data.Dataset.from_tensor_slices((trainX,y_train)).shuffle(trainX.shape[0]).batch(batch_size)
valid_ds = tf.data.Dataset.from_tensor_slices((testX,y_test)).batch(batch_size)

In [103]:
def loss_object1(predictions):
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def one_percentage(predictions):
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer1 = tf.keras.optimizers.Nadam()
optimizer2 = tf.keras.optimizers.RMSprop()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_ones = tf.keras.metrics.Mean(name='train_ones')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_ones = tf.keras.metrics.Mean(name='test_ones')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [104]:
# seperate partial model
alpha = -0.1 #pahse1: -0.1 / 0.0 ; phase2: 0.01~0.05~0.1
@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=False) as tape:
        emb, pred_imp = model1(x)
        #loss1 = alpha*loss_object1(pred_imp) #phase1
        #pred_imp2 = tf.math.round(pred_imp)
        #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
        pred_imp2 = tf.math.round(pred_imp)
        loss1 = alpha*loss_object1(pred_imp) #pahse2
        pred_cat = model2(emb,pred_imp2) #pahse1: pred_imp; phase2; pred_imp2
        loss2 = loss_object2(yc, pred_cat)
        loss = loss1 + loss2
    trainable_variable = model1.trainable_variables
    trainable_variable.extend(model2.trainable_variables)
    gradients = tape.gradient(loss,trainable_variable)
    optimizer1.apply_gradients(zip(gradients,trainable_variable))
    
    train_loss(loss)
    train_accuracy(yc, pred_cat)
    ones = one_percentage(pred_imp2)
    train_ones(ones)
    
@tf.function
def test_step(x,yc):
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    loss1 = alpha*loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #phase1: pred_imp ; phase2:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    #t_loss = loss1 + loss2
    t_loss = loss1 + loss2
    
    test_loss(t_loss)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2)
    test_ones(t_ones)

In [55]:
#AIO
alpha = 0.1
@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=True) as tape: #persistent=True
        pred_imp,pred_round , pred_cat = model(x)
#         pred_cat = model(x)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
        loss1 = alpha*loss_object1(pred_imp)
        loss2 = loss_object2(yc,pred_cat)
        loss = loss_object2(yc,pred_cat)
#     gradients = tape.gradient(loss, model.trainable_variables)
    grad1 = tape.gradient(loss1, model.trainable_variables)
    grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer1.apply_gradients(zip(gradients, model.trainable_variables))
    optimizer1.apply_gradients(zip(grad1, model.trainable_variables))
    optimizer2.apply_gradients(zip(grad2, model.trainable_variables))
#     with tf.GradientTape() as tape:
#         pred_imp , pred_cat = model(x)
#         loss2 = loss_object2(yc,pred_cat)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
#     grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer2.apply_gradients(zip(grad2, model.trainable_variables))

    train_loss(loss)
    train_accuracy(yc, pred_cat)
    ones = one_percentage(pred_round)
    train_ones(ones)
    
@tf.function
def test_step(x,yc):
    pred_imp,pred_round, pred_cat = model(x)
#     pred_cat = model(x)
    t_loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat) 
#     t_loss = loss_object2(yc,pred_cat)
    
    test_loss(t_loss)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_round)
    test_ones(t_ones)

In [105]:
EPOCHS = 5000
gc.collect()
for epoch in range(EPOCHS):
    for text, labels in train_ds:
        train_step(text, labels)

    for test_text, test_labels in valid_ds:
        test_step(test_text, test_labels)

    template = 'Epoch {}, Loss: {},Ones#: {}, Accuracy: {}, Test Loss: {},Test Ones#: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                        train_loss.result(),
                        train_ones.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_ones.result(),
                        test_accuracy.result()*100))

    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_ones.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_ones.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 2.6595335006713867,Ones#: 0.4813397228717804, Accuracy: 37.16706848144531, Test Loss: 2.3049495220184326,Test Ones#: 0.9999974370002747, Test Accuracy: 37.173282623291016
Epoch 2, Loss: 2.215036153793335,Ones#: 1.0, Accuracy: 39.90314865112305, Test Loss: 1.9382696151733398,Test Ones#: 0.9999974370002747, Test Accuracy: 55.32429885864258
Epoch 3, Loss: 1.7228057384490967,Ones#: 0.9970971941947937, Accuracy: 56.65859603881836, Test Loss: 1.5622496604919434,Test Ones#: 0.99491947889328, Test Accuracy: 60.89060974121094
Epoch 4, Loss: 1.471312165260315,Ones#: 0.9943690299987793, Accuracy: 61.416465759277344, Test Loss: 1.461750864982605,Test Ones#: 0.9936204552650452, Test Accuracy: 61.6650505065918
Epoch 5, Loss: 1.3410887718200684,Ones#: 0.9938207864761353, Accuracy: 66.12590789794922, Test Loss: 1.3854622840881348,Test Ones#: 0.9929145574569702, Test Accuracy: 65.00483703613281
Epoch 6, Loss: 1.2183246612548828,Ones#: 0.9937772154808044, Accuracy: 70.0, Test Loss: 1.3327

Epoch 46, Loss: 0.028143342584371567,Ones#: 0.9984986782073975, Accuracy: 95.6295394897461, Test Loss: 1.4247668981552124,Test Ones#: 0.9982450008392334, Test Accuracy: 77.39593505859375
Epoch 47, Loss: 0.026335101574659348,Ones#: 0.9985827207565308, Accuracy: 95.70217895507812, Test Loss: 1.4157193899154663,Test Ones#: 0.998284637928009, Test Accuracy: 76.6698989868164
Epoch 48, Loss: 0.02873094566166401,Ones#: 0.9984918236732483, Accuracy: 95.82324981689453, Test Loss: 1.5407456159591675,Test Ones#: 0.9980921149253845, Test Accuracy: 75.89545440673828
Epoch 49, Loss: 0.03753595054149628,Ones#: 0.9985781311988831, Accuracy: 95.38741302490234, Test Loss: 1.461093783378601,Test Ones#: 0.998220682144165, Test Accuracy: 76.33106994628906
Epoch 50, Loss: 0.03267945349216461,Ones#: 0.9985161423683167, Accuracy: 95.26634216308594, Test Loss: 1.478392481803894,Test Ones#: 0.9982093572616577, Test Accuracy: 75.79864501953125
Epoch 51, Loss: 0.02943595126271248,Ones#: 0.9985562562942505, Accura

Epoch 90, Loss: 0.007322156801819801,Ones#: 0.9993834495544434, Accuracy: 95.75060272216797, Test Loss: 1.6279412508010864,Test Ones#: 0.9993334412574768, Test Accuracy: 76.04065704345703
Epoch 91, Loss: 0.005740508437156677,Ones#: 0.9994795322418213, Accuracy: 95.93220520019531, Test Loss: 1.6301661729812622,Test Ones#: 0.9993463158607483, Test Accuracy: 75.70183563232422
Epoch 92, Loss: 0.006993330083787441,Ones#: 0.9995066523551941, Accuracy: 95.55690002441406, Test Loss: 1.6175506114959717,Test Ones#: 0.9993272423744202, Test Accuracy: 76.6698989868164
Epoch 93, Loss: 0.007765887305140495,Ones#: 0.9995211362838745, Accuracy: 95.78692626953125, Test Loss: 1.6074697971343994,Test Ones#: 0.9992991089820862, Test Accuracy: 75.750244140625
Epoch 94, Loss: 0.011603409424424171,Ones#: 0.9994687438011169, Accuracy: 95.81114196777344, Test Loss: 1.5853605270385742,Test Ones#: 0.9993144869804382, Test Accuracy: 76.08905792236328
Epoch 95, Loss: 0.029879311099648476,Ones#: 0.9994114637374878,

Epoch 134, Loss: 0.0005291042034514248,Ones#: 0.9997662901878357, Accuracy: 96.00484466552734, Test Loss: 1.652835488319397,Test Ones#: 0.9996272921562195, Test Accuracy: 76.9603042602539
Epoch 135, Loss: -0.00046158890472725034,Ones#: 0.9997580647468567, Accuracy: 96.00484466552734, Test Loss: 1.65408456325531,Test Ones#: 0.9996349215507507, Test Accuracy: 76.57308959960938
Epoch 136, Loss: -0.0016204295679926872,Ones#: 0.9997759461402893, Accuracy: 96.07748413085938, Test Loss: 1.6386048793792725,Test Ones#: 0.9996604919433594, Test Accuracy: 76.7667007446289
Epoch 137, Loss: -0.0017621155129745603,Ones#: 0.999789297580719, Accuracy: 95.79902648925781, Test Loss: 1.6653549671173096,Test Ones#: 0.9996781945228577, Test Accuracy: 76.4278793334961
Epoch 138, Loss: -0.001380899571813643,Ones#: 0.999796450138092, Accuracy: 95.94430541992188, Test Loss: 1.637756586074829,Test Ones#: 0.9996859431266785, Test Accuracy: 76.6698989868164
Epoch 139, Loss: -0.0017655572155490518,Ones#: 0.9998065

Epoch 178, Loss: -0.002633350668475032,Ones#: 0.9999780654907227, Accuracy: 95.90798950195312, Test Loss: 1.6729096174240112,Test Ones#: 0.9999080896377563, Test Accuracy: 76.81510162353516
Epoch 179, Loss: -0.0038078385405242443,Ones#: 0.9999900460243225, Accuracy: 95.93220520019531, Test Loss: 1.6898407936096191,Test Ones#: 0.9999105930328369, Test Accuracy: 76.7667007446289
Epoch 180, Loss: -0.0023653365205973387,Ones#: 0.9999927282333374, Accuracy: 96.17433166503906, Test Loss: 1.6730055809020996,Test Ones#: 0.9999105930328369, Test Accuracy: 76.7667007446289
Epoch 181, Loss: -0.003474246710538864,Ones#: 0.9999934434890747, Accuracy: 95.96852111816406, Test Loss: 1.6823370456695557,Test Ones#: 0.9999105930328369, Test Accuracy: 76.28267669677734
Epoch 182, Loss: -0.0031928180251270533,Ones#: 0.9999940395355225, Accuracy: 96.17433166503906, Test Loss: 1.654140591621399,Test Ones#: 0.9999107122421265, Test Accuracy: 76.23426818847656
Epoch 183, Loss: -0.0029999406542629004,Ones#: 0.9

...

Epoch 4758, Loss: -0.012273547239601612,Ones#: 1.0, Accuracy: 96.28329467773438, Test Loss: 2.066044807434082,Test Ones#: 1.0, Test Accuracy: 75.79864501953125
Epoch 4759, Loss: -0.012216616421937943,Ones#: 1.0, Accuracy: 96.38014221191406, Test Loss: 2.0643818378448486,Test Ones#: 1.0, Test Accuracy: 75.750244140625
Epoch 4760, Loss: -0.012390880845487118,Ones#: 1.0, Accuracy: 96.53752899169922, Test Loss: 2.063788414001465,Test Ones#: 1.0, Test Accuracy: 75.94384765625
Epoch 4761, Loss: -0.012535476125776768,Ones#: 1.0, Accuracy: 96.30750274658203, Test Loss: 2.067016839981079,Test Ones#: 1.0, Test Accuracy: 75.8470458984375
Epoch 4762, Loss: -0.012278733775019646,Ones#: 1.0, Accuracy: 96.46488952636719, Test Loss: 2.0646700859069824,Test Ones#: 1.0, Test Accuracy: 75.89545440673828
Epoch 4763, Loss: -0.012197908945381641,Ones#: 1.0, Accuracy: 96.42857360839844, Test Loss: 2.068516731262207,Test Ones#: 1.0, Test Accuracy: 75.79864501953125
Epoch 4764, Loss: -0.012232933193445206,Ones

Epoch 4810, Loss: -0.01233395840972662,Ones#: 1.0, Accuracy: 96.30750274658203, Test Loss: 2.092649459838867,Test Ones#: 1.0, Test Accuracy: 75.07260131835938
Epoch 4811, Loss: -0.012384476140141487,Ones#: 1.0, Accuracy: 96.44068145751953, Test Loss: 2.0915610790252686,Test Ones#: 1.0, Test Accuracy: 75.12100982666016
Epoch 4812, Loss: -0.0119585320353508,Ones#: 1.0, Accuracy: 96.46488952636719, Test Loss: 2.086915969848633,Test Ones#: 1.0, Test Accuracy: 75.1694107055664
Epoch 4813, Loss: -0.012229802086949348,Ones#: 1.0, Accuracy: 96.3559341430664, Test Loss: 2.0836589336395264,Test Ones#: 1.0, Test Accuracy: 75.45982360839844
Epoch 4814, Loss: -0.011777224950492382,Ones#: 1.0, Accuracy: 96.30750274658203, Test Loss: 2.0983073711395264,Test Ones#: 1.0, Test Accuracy: 75.31462097167969
Epoch 4815, Loss: -0.011865731328725815,Ones#: 1.0, Accuracy: 96.48910522460938, Test Loss: 2.1008598804473877,Test Ones#: 1.0, Test Accuracy: 75.21781158447266
Epoch 4816, Loss: -0.01245043519884348,On

Epoch 4862, Loss: -0.00973229669034481,Ones#: 1.0, Accuracy: 96.51332092285156, Test Loss: 2.0976905822753906,Test Ones#: 1.0, Test Accuracy: 74.7337875366211
Epoch 4863, Loss: -0.010134832002222538,Ones#: 1.0, Accuracy: 96.29539489746094, Test Loss: 2.0952999591827393,Test Ones#: 1.0, Test Accuracy: 75.70183563232422
Epoch 4864, Loss: -0.009927763603627682,Ones#: 1.0, Accuracy: 96.27118682861328, Test Loss: 2.0951807498931885,Test Ones#: 1.0, Test Accuracy: 74.97579956054688
Epoch 4865, Loss: -0.01015752088278532,Ones#: 1.0, Accuracy: 96.28329467773438, Test Loss: 2.085768222808838,Test Ones#: 1.0, Test Accuracy: 74.87899017333984
Epoch 4866, Loss: -0.01035330630838871,Ones#: 1.0, Accuracy: 96.33171844482422, Test Loss: 2.097240686416626,Test Ones#: 1.0, Test Accuracy: 75.21781158447266
Epoch 4867, Loss: -0.010136092081665993,Ones#: 1.0, Accuracy: 96.40435791015625, Test Loss: 2.10331654548645,Test Ones#: 1.0, Test Accuracy: 75.21781158447266
Epoch 4868, Loss: -0.010890603996813297,On

Epoch 4914, Loss: -0.006495577748864889,Ones#: 1.0, Accuracy: 96.21065521240234, Test Loss: 2.055389642715454,Test Ones#: 1.0, Test Accuracy: 75.45982360839844
Epoch 4915, Loss: -0.007653102278709412,Ones#: 1.0, Accuracy: 96.13801574707031, Test Loss: 2.0644774436950684,Test Ones#: 1.0, Test Accuracy: 75.41142272949219
Epoch 4916, Loss: -0.008358822204172611,Ones#: 1.0, Accuracy: 96.29539489746094, Test Loss: 2.104257822036743,Test Ones#: 1.0, Test Accuracy: 75.70183563232422
Epoch 4917, Loss: -0.009785161353647709,Ones#: 1.0, Accuracy: 96.46488952636719, Test Loss: 2.0805399417877197,Test Ones#: 1.0, Test Accuracy: 75.2662124633789
Epoch 4918, Loss: -0.009608219377696514,Ones#: 1.0, Accuracy: 96.33171844482422, Test Loss: 2.0718345642089844,Test Ones#: 1.0, Test Accuracy: 75.31462097167969
Epoch 4919, Loss: -0.009892110712826252,Ones#: 1.0, Accuracy: 96.46488952636719, Test Loss: 2.078707695007324,Test Ones#: 1.0, Test Accuracy: 75.60503387451172
Epoch 4920, Loss: -0.01121855061501264

Epoch 4966, Loss: -0.011468999087810516,Ones#: 1.0, Accuracy: 96.31961059570312, Test Loss: 2.0505173206329346,Test Ones#: 1.0, Test Accuracy: 75.65343475341797
Epoch 4967, Loss: -0.007603030186146498,Ones#: 1.0, Accuracy: 96.29539489746094, Test Loss: 2.041471481323242,Test Ones#: 1.0, Test Accuracy: 75.21781158447266
Epoch 4968, Loss: -0.010746764950454235,Ones#: 1.0, Accuracy: 96.13801574707031, Test Loss: 2.0317723751068115,Test Ones#: 1.0, Test Accuracy: 75.750244140625
Epoch 4969, Loss: -0.011032897047698498,Ones#: 1.0, Accuracy: 96.19854736328125, Test Loss: 2.030139446258545,Test Ones#: 1.0, Test Accuracy: 75.70183563232422
Epoch 4970, Loss: -0.011733468621969223,Ones#: 1.0, Accuracy: 96.44068145751953, Test Loss: 2.0242373943328857,Test Ones#: 1.0, Test Accuracy: 75.70183563232422
Epoch 4971, Loss: -0.01213131658732891,Ones#: 1.0, Accuracy: 96.25907897949219, Test Loss: 2.029296636581421,Test Ones#: 1.0, Test Accuracy: 75.750244140625
Epoch 4972, Loss: -0.011919067241251469,On

* 不用sigmoid或是hard_sigmoid，改成relu(linear)拆成兩個model，把round前面多加上clip
    * sigmoid中間的變化太快(一瞬間就會掉到0或是1)，改成relu在>0~無限大(linear為了還在0~1)再去clip再round，可以看到每個epoch的變化
* 若設定alpha，就像是regularizer term (penalty)，設越大drop越多
* 一開始先很多ones，再越來越少個

* 同一個opt若加入transform就會train不起來
* 兩個不同的opt加入transform也會train不起來 (persistent、non-persis都不行)，且與BN無關